In [1]:
from qwen2.modeling_qwen2 import Qwen2MLAForCausalLM
from transformers import AutoTokenizer
import torch
from copy import deepcopy
from tqdm import tqdm

/root/miniconda3/envs/transmla/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
model = Qwen2MLAForCausalLM.from_pretrained("/root/mfx/huggingface/Qwen/Qwen2.5-3B", device_map='cuda:4', attn_implementation="eager", partial_rotary_factor=1, rope_repeat=True)
tokenizer = AutoTokenizer.from_pretrained("/root/mfx/huggingface/Qwen/Qwen2.5-3B")
model

Loading checkpoint shards: 100%|██████████| 2/2 [00:05<00:00,  2.67s/it]
Some weights of Qwen2MLAForCausalLM were not initialized from the model checkpoint at /root/mfx/huggingface/Qwen/Qwen2.5-3B and are newly initialized: ['model.layers.0.self_attn.k_up_proj.weight', 'model.layers.0.self_attn.v_up_proj.weight', 'model.layers.1.self_attn.k_up_proj.weight', 'model.layers.1.self_attn.v_up_proj.weight', 'model.layers.10.self_attn.k_up_proj.weight', 'model.layers.10.self_attn.v_up_proj.weight', 'model.layers.11.self_attn.k_up_proj.weight', 'model.layers.11.self_attn.v_up_proj.weight', 'model.layers.12.self_attn.k_up_proj.weight', 'model.layers.12.self_attn.v_up_proj.weight', 'model.layers.13.self_attn.k_up_proj.weight', 'model.layers.13.self_attn.v_up_proj.weight', 'model.layers.14.self_attn.k_up_proj.weight', 'model.layers.14.self_attn.v_up_proj.weight', 'model.layers.15.self_attn.k_up_proj.weight', 'model.layers.15.self_attn.v_up_proj.weight', 'model.layers.16.self_attn.k_up_proj.weight

Qwen2MLAForCausalLM(
  (model): Qwen2MLAModel(
    (embed_tokens): Embedding(151936, 2048)
    (layers): ModuleList(
      (0-35): 36 x Qwen2DecoderLayer(
        (self_attn): Qwen2MLAttention(
          (k_proj): Linear(in_features=2048, out_features=256, bias=True)
          (v_proj): Linear(in_features=2048, out_features=256, bias=True)
          (q_proj): Linear(in_features=2048, out_features=2048, bias=True)
          (o_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (k_up_proj): Linear(in_features=256, out_features=2048, bias=False)
          (v_up_proj): Linear(in_features=256, out_features=2048, bias=False)
          (rotary_emb): Qwen2RotaryEmbedding()
        )
        (mlp): Qwen2MLP(
          (gate_proj): Linear(in_features=2048, out_features=11008, bias=False)
          (up_proj): Linear(in_features=2048, out_features=11008, bias=False)
          (down_proj): Linear(in_features=11008, out_features=2048, bias=False)
          (act_fn): SiLU()
    

In [3]:
hidden_size = model.config.hidden_size
n_heads = model.config.num_attention_heads
kv_heads = model.config.num_key_value_heads
head_dim = model.config.hidden_size//model.config.num_attention_heads
latent_dim = kv_heads * head_dim
kv_groups = model.config.num_attention_heads // model.config.num_key_value_heads
model.config.partial_rotary_factor

1

In [4]:
# Insert identity matrices
for name,module in model.named_modules():
    if 'k_up_proj' in name or "v_up_proj" in name:
        module.weight.data = torch.stack([torch.eye(latent_dim).reshape(kv_heads, head_dim, latent_dim)]*kv_groups,dim=1).reshape(hidden_size, latent_dim).contiguous().to(module.weight.data.device,module.weight.data.dtype)

In [5]:
output = model.generate(**tokenizer("给我讲一个故事吧",return_tensors="pt").to("cuda:4"), max_new_tokens=500, do_sample=False)
print(tokenizer.batch_decode(output)[0])

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


给我讲一个故事吧。
当然可以，这是一个关于勇气和友谊的故事。

从前，在一个遥远的森林里，住着一只名叫蓝羽的小鸟。蓝羽拥有一身美丽的蓝色羽毛，但它总是感到孤独，因为它没有朋友。每天，它都在森林中飞来飞去，寻找可以交朋友的伙伴。

一天，蓝羽遇到了一只受伤的小兔子，名叫米洛。米洛的腿受伤了，无法行走。蓝羽立刻停下来，用它的翅膀轻轻地拍打着地面，帮助米洛站起来。米洛感激地看着蓝羽，说：“谢谢你，蓝羽。你真是个好伙伴。”

从那天起，蓝羽和米洛成了最好的朋友。他们一起探索森林，分享彼此的故事和秘密。蓝羽教会了米洛如何飞翔，而米洛则教给了蓝羽如何照顾受伤的动物。

随着时间的推移，森林里的其他动物也开始注意到蓝羽和米洛之间的特殊关系。它们邀请蓝羽和米洛参加各种活动，包括音乐会、舞蹈比赛和野餐。蓝羽和米洛都非常开心，因为他们知道，有了朋友，生活变得更加美好。

然而，好景不长，森林里来了一只凶猛的狼，它威胁着所有动物的安全。蓝羽和米洛决定联手对抗这只狼。他们利用自己的智慧和勇气，成功地将狼赶走了。

从那以后，蓝羽和米洛成为了森林中的英雄。他们的友谊证明了，即使是最小的生物也能拥有巨大的力量和勇气。而蓝羽也明白了，真正的友谊是无价的，它能够带来快乐、支持和力量。

这个故事告诉我们，无论我们多么渺小，只要我们有勇气去帮助别人，就有机会找到真正的友谊和幸福。<|endoftext|>


In [6]:
for name,module in model.named_modules():
    if name.endswith("self_attn"):
        # Orthogonal q_proj and k_up_proj
        k_up_weight = deepcopy(module.k_up_proj.weight.data).reshape(n_heads, head_dim, latent_dim) # (n_heads, head_dim, latent_dim)
        q_weight = deepcopy(module.q_proj.weight.data).reshape(n_heads, head_dim, hidden_size) # (n_heads, head_dim, hidden_size)
        if module.q_proj.bias is not None:
            q_weight = torch.cat([q_weight,deepcopy(module.q_proj.bias.data).reshape(n_heads, head_dim, 1)],dim=-1)
        q_k_up = torch.einsum("hdc,hdD->hcD",k_up_weight, q_weight) # (n_heads, latent_dim, hidden_size), rank<=head_dim
        U,S,V = torch.svd_lowrank(q_k_up, head_dim, niter=16) # U(n_heads, latent_dim, head_dim), S(n_heads, head_dim), V(n_heads, hidden_size, head_dim)
        US_sqrt = torch.einsum('hLd,hd->hdL',U,torch.sqrt(S)) # (n_heads, head_dim, latent_dim)
        S_sqrtV = torch.einsum('hd,hDd->hdD',torch.sqrt(S),V) # (n_heads, head_dim, hidden_size)
        if module.q_proj.bias is not None:
            module.q_proj.bias.data = S_sqrtV[:,:,-1].reshape(-1).contiguous()
            S_sqrtV = S_sqrtV[:,:,:-1]
        module.k_up_proj.weight.data = US_sqrt.reshape(n_heads*head_dim, latent_dim).contiguous()
        module.q_proj.weight.data = S_sqrtV.reshape(n_heads*head_dim, hidden_size).contiguous()

        # Orthogonal o_proj and v_up_proj
        v_up_weight = deepcopy(module.v_up_proj.weight.data).reshape(n_heads, head_dim, latent_dim)
        o_weight = deepcopy(module.o_proj.weight.data).reshape(hidden_size, n_heads, head_dim)
        v_up_o = torch.einsum("hdc,Dhd->hcD",v_up_weight, o_weight) # (n_heads, latent_dim, hidden_size), rank<=head_dim
        U,S,V = torch.svd_lowrank(v_up_o, head_dim, niter=16) # U(n_heads, latent_dim, head_dim), S(n_heads, head_dim), V(n_heads, hidden_size, head_dim)
        US_sqrt = torch.einsum('hLd,hd->hdL',U,torch.sqrt(S)) # (n_heads, head_dim, latent_dim)
        S_sqrtV = torch.einsum('hd,hDd->Dhd',torch.sqrt(S),V) # (hidden_size, n_heads, head_dim)
        module.v_up_proj.weight.data = US_sqrt.reshape(hidden_size, latent_dim).contiguous()
        module.o_proj.weight.data = S_sqrtV.reshape(hidden_size, n_heads*head_dim).contiguous()

In [7]:
output = model.generate(**tokenizer("给我讲一个故事吧",return_tensors="pt").to("cuda:4"), max_new_tokens=500, do_sample=False)
print(tokenizer.batch_decode(output)[0])

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


给我讲一个故事吧。
当然可以，这是一个关于勇气和友谊的故事。

从前，在一个遥远的森林里，住着一只名叫蓝羽的小鸟。蓝羽拥有一身美丽的蓝色羽毛，但它总是感到孤独，因为它没有朋友。每天，它都在森林中飞来飞去，寻找可以交朋友的伙伴。

一天，蓝羽遇到了一只受伤的小兔子，名叫米洛。米洛的腿受伤了，无法行走。蓝羽立刻停下来，用它的翅膀轻轻地拍打着地面，帮助米洛站起来。米洛感激地看着蓝羽，说：“谢谢你，蓝羽。你真是个好伙伴。”

从那天起，蓝羽和米洛成了最好的朋友。他们一起探索森林，分享彼此的故事和秘密。蓝羽教会了米洛如何飞翔，而米洛则教给了蓝羽如何照顾受伤的动物。

随着时间的推移，森林里的其他动物也开始注意到蓝羽和米洛之间的特殊关系。它们邀请蓝羽和米洛参加各种活动，包括音乐会、舞蹈比赛和野餐。蓝羽和米洛都非常开心，因为他们知道，有了朋友，生活变得更加美好。

然而，好景不长，森林里来了一只凶猛的狼，它威胁着所有动物的安全。蓝羽和米洛决定联手对抗这只狼。他们利用自己的智慧和勇气，成功地将狼赶走了。

从那以后，蓝羽和米洛成为了森林中的英雄。他们的友谊证明了，即使是最小的生物也能拥有巨大的力量和勇气。而蓝羽也明白了，真正的友谊是无价的，它能够带来快乐、支持和力量。

这个故事告诉我们，无论我们多么渺小，只要我们有勇气去帮助别人，就有机会找到真正的友谊和幸福。<|endoftext|>


In [8]:
for name,module in model.named_modules():
    if name.endswith("self_attn"):
        # Absorb k_up_proj into q_proj
        k_up_weight = deepcopy(module.k_up_proj.weight.data).reshape(n_heads, head_dim, latent_dim) # (n_heads, head_dim, latent_dim)
        q_weight = deepcopy(module.q_proj.weight.data).reshape(n_heads, head_dim, hidden_size) # (n_heads, head_dim, hidden_size)
        if module.q_proj.bias is not None:
            q_weight = torch.cat([q_weight,deepcopy(module.q_proj.bias.data).reshape(n_heads, head_dim, 1)],dim=-1)
        q_k_up = torch.einsum("hdc,hdD->hcD",k_up_weight, q_weight) # (n_heads, latent_dim, hidden_size), rank<=head_dim
        q_proj = torch.nn.Linear(hidden_size, n_heads*latent_dim, bias=(module.q_proj.bias is not None))
        q_proj = q_proj.to(device=module.q_proj.weight.device, dtype=module.q_proj.weight.dtype)
        if module.q_proj.bias is not None:
            q_proj.bias.data = q_k_up[:,:,-1].reshape(-1).contiguous()
            q_k_up = q_k_up[:,:,:-1]
        q_proj.weight.data = q_k_up.reshape(n_heads*latent_dim, hidden_size).contiguous()
        setattr(module, "q_proj", q_proj)
        delattr(module, "k_up_proj")
        # Absorb v_up_proj into o_proj
        v_up_weight = deepcopy(module.v_up_proj.weight.data).reshape(n_heads, head_dim, latent_dim) # (n_heads, head_dim, latent_dim)
        o_weight = deepcopy(module.o_proj.weight.data).reshape(hidden_size, n_heads, head_dim) # (n_heads, head_dim, hidden_size)
        v_up_o = torch.einsum("hdc,Dhd->Dhc",v_up_weight, o_weight) # (hidden_size, n_heads, latent_dim), rank<=head_dim
        o_proj = torch.nn.Linear(n_heads*latent_dim, hidden_size, bias=(module.o_proj.bias is not None))
        o_proj = o_proj.to(device=module.o_proj.weight.device, dtype=module.o_proj.weight.dtype)
        o_proj.weight.data = v_up_o.reshape(hidden_size, n_heads*latent_dim).contiguous()
        if module.o_proj.bias is not None:
            o_proj.bias.data = module.o_proj.bias
        setattr(module, "o_proj", o_proj)
        delattr(module, "v_up_proj")
        module.absorb = True

In [9]:
output = model.generate(**tokenizer("给我讲一个故事吧",return_tensors="pt").to("cuda:4"), max_new_tokens=500, do_sample=False)
print(tokenizer.batch_decode(output)[0])

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


给我讲一个故事吧。
当然可以，这是一个关于勇气和友谊的故事。

从前，在一个遥远的森林里，住着一只名叫蓝羽的小鸟。蓝羽拥有一身美丽的蓝色羽毛，但它总是感到孤独，因为它没有朋友。每天，它都在森林中飞来飞去，寻找可以交朋友的伙伴。

一天，蓝羽遇到了一只受伤的小兔子，名叫米洛。米洛的腿受伤了，无法行走。蓝羽立刻停下来，用它的翅膀轻轻地拍打着地面，帮助米洛站起来。米洛感激地看着蓝羽，说：“谢谢你，蓝羽。你真是个好伙伴。”

从那天起，蓝羽和米洛成了最好的朋友。他们一起探索森林，分享彼此的故事和秘密。蓝羽教会了米洛如何飞翔，而米洛则教给了蓝羽如何照顾受伤的动物。

随着时间的推移，森林里的其他动物也开始注意到蓝羽和米洛之间的特殊关系。它们邀请蓝羽和米洛参加各种活动，包括音乐会、舞蹈比赛和野餐。蓝羽和米洛都非常开心，因为他们知道，有了朋友，生活变得更加美好。

然而，好景不长，森林里来了一只凶猛的狼，它威胁着所有动物的安全。蓝羽和米洛决定联手对抗这只狼。他们利用自己的智慧和勇气，成功地将狼赶走了。

从那以后，蓝羽和米洛成为了森林中的英雄。他们的友谊证明了，即使是最小的生物也能拥有巨大的力量和勇气。而蓝羽也明白了，真正的友谊是无价的，它能够带来快乐、支持和力量。

这个故事告诉我们，无论我们多么渺小，只要我们有勇气去帮助别人，就有机会找到真正的友谊和幸福。<|endoftext|>
